# Play Atari games with DQN

This notebook shows how to play image based Atari games (will leave out the RAM games since the pipeline can be very different) with DQN agents.

In [1]:
import gym
import numpy as np
import tensorflow as tf
from collections import deque
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tf_agents.replay_buffers.tf_uniform_replay_buffer import TFUniformReplayBuffer

In [2]:
def play_episode(env, agent, epsilon=0.0, collect=False, buffer=None, render=False, max_steps=100000):
    state = env.reset()
    done = False
    steps = 0
    while not done and steps < max_steps:
        if render:
            env.render()
        action = agent.collect_policy(state, epsilon)
        next_state, reward, done, info = env.step(action)
        q_vals = agent.get_q_vals(state).numpy()[0]
        next_q_vals = agent.get_next_q_vals(next_state).numpy()[0]
        q_vals[action] = reward
        if not done:
            q_vals[action] += next_q_vals.max()
        buffer.add_batch((
            tf.convert_to_tensor([state], dtype=tf.float32),
            tf.convert_to_tensor([q_vals], dtype=tf.float32)
        ))
        state = next_state
        steps += 1

In [3]:
class Qnet(Model):
    def __init__(self, action_space_size, state_shape):
        super(Qnet, self).__init__()
        self.action_space_size = action_space_size
        self.state_shape = state_shape
        # Define conv layers
        self.conv_in = Conv2D(16, 8, strides=(4, 4), activation='relu', input_shape=self.state_shape)
        self.conv_0 = Conv2D(32, 4, strides=(2, 2), activation='relu')
        self.flatten = Flatten()
        # Dense layers
        self.dense_0 = Dense(256, activation='relu')
        self.dense_out = Dense(self.action_space_size, activation='linear')
    
    def call(self, inputs):
        x = inputs
        x = self.conv_in(x)
        x = self.conv_0(x)
        x = self.flatten(x)
        x = self.dense_0(x)
        x = self.dense_out(x)
        return x

In [4]:
class ReplayBuffer:
    def __init__(self, max_length, state_shape, action_space_size):
        self.max_length = max_length
        self.state_shape = state_shape
        self.action_space_size = action_space_size
        self.state_spec = tf.TensorSpec(state_shape, dtype=tf.float32, name='state')
        self.q_vals_spec = tf.TensorSpec([self.action_space_size], dtype=tf.float32, name='q_vals')
        self.buffer = TFUniformReplayBuffer(
            data_spec=(self.state_spec, self.q_vals_spec),
            batch_size=1,
            max_length=self.max_length)
    
    def add_batch(self, batch):
        self.buffer.add_batch(batch)
    
    def get_next(self):
        return self.buffer.get_next(sample_batch_size=32)

In [5]:
class DqnAgent:    
    def __init__(self, action_space_size, state_shape, train_per_eps, learning_rate):
        self.action_space_size = action_space_size
        self.state_shape = state_shape
        self.learning_rate = learning_rate
        self.train_per_eps = train_per_eps
        self.q_net = self.create_q_net()
        self.target_q_net = self.create_q_net()
        
    def create_q_net(self):
        q_net = Qnet(self.action_space_size, self.state_shape)
        q_net.compile(optimizer=tf.optimizers.Adam(learning_rate=self.learning_rate),
                      loss='mse')
        return q_net
    
    def random_policy(self, state):
        return np.random.randint(self.action_space_size)
    
    def net_policy(self, state):
        q_vals = self.get_q_vals(state).numpy()[0]
        action = np.argmax(q_vals)
        return action
    
    def collect_policy(self, state, epsilon):
        if np.random.random() < epsilon:
            return self.random_policy(state)
        return self.net_policy(state)
    
    def get_next_q_vals(self, state):
        next_q_vals = self.target_q_net(tf.convert_to_tensor([state], dtype=tf.float32))
        return next_q_vals
    
    def get_q_vals(self, state):
        q_vals = self.q_net(tf.convert_to_tensor([state], dtype=tf.float32))
        return q_vals
    
    def train(self, replay_buffer):
        loss = []
        for _ in range(self.train_per_eps):
            batch = replay_buffer.get_next()
            print('states are:')
            print(batch[0])
            print('target q vals are:')
            print(batch[1])
            result = self.q_net.fit(x=batch[0], y=batch[1], verbose=1)
            loss.append(result.history['loss'])
        return loss

In [6]:
class GameEnv:
    """
    Although OpenAI gym library provides most of the
    things we need, the state and reward it emits is
    too primative to be used. To keep the state and
    reward conversion consistent, we wrap the OpenAI
    gym environment in a shim class.
    """
    @staticmethod
    def sanitize_state(state, downsample=2):
        img = Image.fromarray(state).convert('L')
        w, h = img.size
        w, h = w // 2, h // 2
        img = np.array(img.resize((w, h)))
        normalized_img = img / 255.0
        return normalized_img
    
    @staticmethod
    def normalize_reward(reward):
        return reward
    
    def __init__(self, game_id, n_frames):
        self.game_id = game_id
        self.n_frames = n_frames
        self.env = gym.make(game_id)
        self.frames = deque(maxlen=self.n_frames)
        
    def get_action_space_size(self):
        return self.env.action_space.n
    
    def get_state_shape(self):
        initial_state = self.reset()
        return initial_state.shape
        
    def render(self):
        self.env.render()
        
    def get_frames(self):
        frames = np.array(self.frames)
        frames = np.swapaxes(frames, 0, 2)
        return frames
        
    def reset(self):
        unsanitized_initial_state = self.env.reset()
        initial_state = self.sanitize_state(unsanitized_initial_state)
        for _ in range(self.n_frames):
            self.frames.append(initial_state)
        return self.get_frames()
    
    def close(self):
        self.env.close()
    
    def step(self, action):
        unsanitized_state, unnormalized_reward, done, info = self.env.step(action)
        state = self.sanitize_state(unsanitized_state)
        self.frames.append(state)
        reward = self.normalize_reward(unnormalized_reward)
        return self.get_frames(), reward, done, info

In [7]:
def init_tensorflow_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
    print('GPU initialized')

In [8]:
def train_agent(max_eps=1, max_steps=100, render=False,
                max_buffer_len=1200, batch_size=64,
                train_per_eps=1, learning_rate=1e-3):
    init_tensorflow_gpu()
    env = GameEnv('Breakout-v0', 4)
    state_shape = env.get_state_shape()
    action_space_size = env.get_action_space_size()
    buf = ReplayBuffer(max_buffer_len, state_shape, action_space_size)
    print('Initialize agent with state shape {0} and action space size {1}'.format(state_shape, action_space_size))
    agent = DqnAgent(action_space_size, state_shape, train_per_eps, learning_rate)
    for eps in range(max_eps):
        play_episode(env, agent, render=render,
                     max_steps=max_steps, collect=True,
                     buffer=buf)
        agent.train(buf)
    env.close()
    print('Done')

In [9]:
train_agent(render=True)

1 Physical GPUs, 1 Logical GPUs
GPU initialized
Initialize agent with state shape (80, 105, 4) and action space size 4
states are:
(<tf.Tensor: shape=(32, 80, 105, 4), dtype=float32, numpy=
array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        ...,

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
   

ValueError: in user code:

    C:\Users\tianh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-3-6d6e53e68ba8>:16 call  *
        x = self.conv_in(x)
    C:\Users\tianh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\base_layer.py:976 __call__  **
        self.name)
    C:\Users\tianh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\input_spec.py:158 assert_input_compatibility
        ' input tensors. Inputs received: ' + str(inputs))

    ValueError: Layer conv2d expects 1 inputs, but it received 2 input tensors. Inputs received: [<tf.Tensor 'IteratorGetNext:0' shape=(32, 80, 105, 4) dtype=float32>, <tf.Tensor 'IteratorGetNext:1' shape=(32, 4) dtype=float32>]
